# YT Comments analysis

In [3]:
from googleapiclient.discovery import build
import pandas as pd
from api import API_KEY

channel_id = "UCWeg2Pkate69NFdBeuRFTAw" #Squeezie channel

youtube = build('youtube', 'v3', developerKey=API_KEY)

In [10]:
def get_channel_stats(youtube, channel_id):
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=channel_id
    )
    response = request.execute()
    data = {
        "channel_name": response['items'][0]['snippet']['title'],
        "channel_id": response['items'][0]['id'],
        "stats": response['items'][0]['statistics']
    }
    return data

In [11]:
get_channel_stats(youtube, channel_id)

{'channel_name': 'SQUEEZIE',
 'channel_id': 'UCWeg2Pkate69NFdBeuRFTAw',
 'stats': {'viewCount': '10428189006',
  'subscriberCount': '18700000',
  'hiddenSubscriberCount': False,
  'videoCount': '1617'}}